In [1]:
# Chargement package
import pandas as pd
import numpy as np

In [ ]:
# Connexion à ggdrive
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [71]:
# Import du fichier via ggdrive
#file_path = "/content/drive/MyDrive/M2_SISE/Python_M2/df_clean.csv" # Lien perso ggdrive
train = pd.read_csv('train_clean.csv', sep = ',')
train = train.drop(columns=['Unnamed: 0'])


print(train.shape)


(6804, 54)


In [4]:
X = train[train.columns[1:-1]] # On enlève la colonne ID et la colonne CLASSE
y = train.match # Colonne target

print(X.shape)
print(y.shape)

(6804, 52)
(6804,)


In [5]:
# Reconvertir les var categ 
ls_categ = ['gender','condtn','samerace','race_o','field_cd','race','goal','date','go_out','career_c'] 

for i in ls_categ:
    X[i] = X[i].astype("category")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6804 entries, 0 to 6803
Data columns (total 52 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   gender    6804 non-null   category
 1   condtn    6804 non-null   category
 2   samerace  6804 non-null   category
 3   race_o    6804 non-null   category
 4   field_cd  6804 non-null   category
 5   race      6804 non-null   category
 6   goal      6804 non-null   category
 7   date      6804 non-null   category
 8   go_out    6804 non-null   category
 9   career_c  6804 non-null   category
 10  int_corr  6804 non-null   float64 
 11  age_o     6804 non-null   float64 
 12  pf_o_att  6804 non-null   float64 
 13  pf_o_sin  6804 non-null   float64 
 14  pf_o_int  6804 non-null   float64 
 15  pf_o_fun  6804 non-null   float64 
 16  pf_o_amb  6804 non-null   float64 
 17  pf_o_sha  6804 non-null   float64 
 18  attr_o    6804 non-null   float64 
 19  sinc_o    6804 non-null   float64 
 20  intel_o 

### Import packages - Sélection Variables

In [7]:
from sklearn.feature_selection import chi2, f_classif, SelectFwe


### Sélection des variables catégorielles

In [8]:
X_categ = X.select_dtypes(include = 'category')
X_categ.head(2)

,gender,condtn,samerace,race_o,field_cd,race,goal,date,go_out,career_c
0,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,1.0
1,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,1.0


In [28]:
sel = SelectFwe(chi2, alpha=0.001)

In [53]:
sel.fit(X_categ, y)

SelectFwe(alpha=0.001, score_func=<function chi2 at 0x7f55298dc680>)

In [54]:
sel.scores_

array([1.48624207e-03, 1.99836605e+00, 4.10322188e-03, 4.68807524e+00,
       6.50995271e+00, 2.46655701e+01, 3.86223760e+00, 1.19693680e+01,
       3.78704032e-01, 2.10653650e+01])

In [55]:
sel.pvalues_

array([9.69247724e-01, 1.57468877e-01, 9.48925321e-01, 3.03726516e-02,
       1.07272361e-02, 6.81915858e-07, 4.93844255e-02, 5.40822827e-04,
       5.38297634e-01, 4.43879119e-06])

In [62]:
print("Avec pvalue < 0.01, 4 variables conservées")
print(X_categ.columns[sel.pvalues_ < 0.01])  

print("Avec pvalue < 0.001 2 variables conservées")
print(X_categ.columns[sel.pvalues_ < 0.001]) 

Avec pvalue < 0.01, 4 variables conservées
Index(['race', 'date', 'career_c'], dtype='object')
Avec pvalue < 0.001 2 variables conservées
Index(['race', 'date', 'career_c'], dtype='object')


In [66]:
col_categ = X_categ.columns[sel.pvalues_ < 0.001].tolist()
col_categ

['race', 'date', 'career_c']

### Sélection variables quantitatives

In [37]:
X_quanti = X.select_dtypes(exclude = 'category')
X_quanti.head(2)

,int_corr,age_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,attr_o,sinc_o,...,music,shopping,yoga,exphappy,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1
0,48.0,29.0,20.0,10.0,50.0,5.0,10.0,5.0,3.0,10.0,...,5.0,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0
1,31.0,23.0,14.0,15.0,16.0,17.0,18.0,20.0,3.0,9.0,...,5.0,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0


In [41]:
selQ = SelectFwe(f_classif, alpha=0.001)

In [58]:
selQ.fit(X_quanti, y)

SelectFwe(alpha=0.001)

In [59]:
selQ.scores_

array([4.22112481e+00, 1.46007442e+01, 1.85012206e-01, 2.99423215e-01,
       3.96892783e-02, 3.71092675e-02, 2.96503723e-01, 1.57462127e+00,
       4.68641209e+02, 1.92174092e+02, 1.94457525e+02, 5.19915625e+02,
       1.22865384e+02, 4.34334352e+02, 1.46876828e+01, 2.54068548e+01,
       6.98852105e+00, 3.31869434e+00, 3.72657281e+00, 2.91263214e-01,
       3.34498942e-02, 3.62431986e+00, 1.88793190e-01, 4.92736481e+00,
       4.57672519e+00, 5.07520330e-01, 1.77478268e+01, 1.77939136e+00,
       1.53101867e+00, 2.85160274e-01, 3.11140872e+00, 1.24745944e+01,
       1.10316010e+01, 1.28164491e+00, 6.22450983e+00, 8.14181569e+00,
       1.53660622e+00, 1.42235352e+00, 1.00286095e+00, 8.41794318e-01,
       2.23153678e+00, 2.21640009e+00])

In [60]:
selQ.pvalues_

array([3.99618068e-002, 1.34046170e-004, 6.67114376e-001, 5.84262191e-001,
       8.42095295e-001, 8.47248501e-001, 5.86100470e-001, 2.09580273e-001,
       1.47374314e-100, 4.09675835e-043, 1.34225033e-043, 5.85623592e-111,
       2.60525997e-028, 1.47837893e-093, 1.28016222e-004, 4.76257601e-007,
       8.22222924e-003, 6.85401054e-002, 5.35946889e-002, 5.89429834e-001,
       8.54887330e-001, 5.69830025e-002, 6.63936721e-001, 2.64674480e-002,
       3.24445887e-002, 4.76238696e-001, 2.55463262e-005, 1.82268743e-001,
       2.16002338e-001, 5.93356216e-001, 7.77905729e-002, 4.15258660e-004,
       9.00439216e-004, 2.57633324e-001, 1.26229084e-002, 4.33865620e-003,
       2.15166436e-001, 2.33058015e-001, 3.16654853e-001, 3.58916628e-001,
       1.35265680e-001, 1.36597491e-001])

In [65]:
print("Avec pvalue < 0.01,",len(X_quanti.columns[selQ.pvalues_ < 0.01]), "variables conservées")
print(X_quanti.columns[selQ.pvalues_ < 0.01], "\n")  
print("Avec pvalue < 0.001",len(X_quanti.columns[selQ.pvalues_ < 0.001]) ,"variables conservées")
print(X_quanti.columns[selQ.pvalues_ < 0.001]) 

Avec pvalue < 0.01, 14 variables conservées
Index(['age_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o',
       'age', 'imprace', 'imprelig', 'clubbing', 'concerts', 'music',
       'exphappy'],
      dtype='object') 

Avec pvalue < 0.001 12 variables conservées
Index(['age_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o',
       'age', 'imprace', 'clubbing', 'concerts', 'music'],
      dtype='object')


In [64]:
col_quanti = X_quanti.columns[selQ.pvalues_ < 0.001].tolist()
col_quanti

['age_o',
 'attr_o',
 'sinc_o',
 'intel_o',
 'fun_o',
 'amb_o',
 'shar_o',
 'age',
 'imprace',
 'clubbing',
 'concerts',
 'music']

### Export de train avec les variables conservées

In [68]:
column_export = ['iid_pid']+ col_categ + col_quanti + ['match']
column_export

['iid_pid',
 'race',
 'date',
 'career_c',
 'age_o',
 'attr_o',
 'sinc_o',
 'intel_o',
 'fun_o',
 'amb_o',
 'shar_o',
 'age',
 'imprace',
 'clubbing',
 'concerts',
 'music',
 'match']

In [69]:
export = train[column_export]
export.shape

(6804, 17)

In [70]:
from google.colab import files
export.to_csv('model_data.csv', index = False)
files.download('model_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>